In [1]:
import pandas as pd

In [2]:
import numpy as np

In [4]:
import janitor

In [11]:
ships = pd.read_excel("seabirds.xls", sheet_name="Ship data by record ID")

In [58]:
birds = pd.read_excel("seabirds.xls", sheet_name="Bird data by record ID")

In [59]:
birds = birds.clean_names()

In [100]:
ships = ships.clean_names()

In [60]:
birds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49019 entries, 0 to 49018
Data columns (total 26 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   record                                                  49019 non-null  int64  
 1   record_id                                               49019 non-null  int64  
 2   species_common_name_taxon_[age_sex_plumage_phase]_      49019 non-null  object 
 3   species_scientific_name_taxon_[age_sex_plumage_phase]_  47918 non-null  object 
 4   species_abbreviation                                    48328 non-null  object 
 5   age                                                     10182 non-null  object 
 6   wanplum                                                 9575 non-null   float64
 7   plphase                                                 81 non-null     object 
 8   sex                                 

In [104]:
ships.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12310 entries, 0 to 12309
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   record     12310 non-null  int64         
 1   record_id  12310 non-null  int64         
 2   date       12310 non-null  datetime64[ns]
 3   time       12308 non-null  object        
 4   lat        12300 non-null  float64       
 5   long       12299 non-null  float64       
 6   ew         12299 non-null  object        
 7   sact       12310 non-null  int64         
 8   speed      7832 non-null   float64       
 9   sdir       1255 non-null   float64       
 10  cld        7523 non-null   float64       
 11  prec       7507 non-null   float64       
 12  wspeed     7622 non-null   float64       
 13  wdir       1292 non-null   float64       
 14  atmp       1036 non-null   float64       
 15  aprs       770 non-null    float64       
 16  sste       7559 non-null   float64      

In [61]:
birds = birds.drop(columns = ["age", "wanplum", "plphase", "sex"])

In [62]:
birds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49019 entries, 0 to 49018
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   record                                                  49019 non-null  int64  
 1   record_id                                               49019 non-null  int64  
 2   species_common_name_taxon_[age_sex_plumage_phase]_      49019 non-null  object 
 3   species_scientific_name_taxon_[age_sex_plumage_phase]_  47918 non-null  object 
 4   species_abbreviation                                    48328 non-null  object 
 5   count                                                   46320 non-null  float64
 6   nfeed                                                   22571 non-null  float64
 7   ocfeed                                                  48328 non-null  object 
 8   nsow                                

In [41]:
#birds.loc[:, birds.columns.str.contains("oc")] = np.where(birds.loc[:, birds.columns.str.contains("oc")] == "U", 0, birds.loc[:, birds.columns.str.contains("oc")] )

In [42]:
#birds.loc[:, birds.columns.str.contains("oc")] = np.where(birds.loc[:, birds.columns.str.contains("oc")] == "Y", 1, 0)

In [52]:
#birds["sightings_total"]= birds.loc[:, birds.columns.str.contains("oc")].sum(axis=1)

In [65]:
birds = birds.rename(columns={"species_common_name_taxon_[age_sex_plumage_phase]_":"common_name",
                      "species_scientific_name_taxon_[age_sex_plumage_phase]_": "scientific_name",})
#shorten the name of two columns

1) Which bird had the most individual sightings?

In [87]:
(birds.groupby(["common_name", "scientific_name", "species_abbreviation"]).agg({"record_id":"count"})
.sort_values(by="record_id", ascending=False)).rename(columns={"record_id":"count"}).reset_index().head(1)

,common_name,scientific_name,species_abbreviation,count
0,Wandering albatross sensu lato PL2,Diomedea antipodensis / exulans PL2,DIOANTEXU PL2,3104


2) Which bird had the highest total count?

In [98]:
(birds.groupby(["common_name", "scientific_name", "species_abbreviation"]).agg({"count":"sum"})
.reset_index().sort_values(by="count", ascending=False).head(1))

,common_name,scientific_name,species_abbreviation,count
218,Short-tailed shearwater,Puffinus tenuirostris,PUFTEN,982553.0


3) Which bird had the highest total count above a latitude of -30?

In [103]:
#need to merge ships and bird info on record_id
birds_ship = birds.merge(ships, on="record_id")

In [107]:
birds_ship = birds_ship.sort_values(by="lat")

In [109]:
birds_ship_lat = birds_ship[birds_ship["lat"]> -30] #filter where lat > -30

In [112]:
(birds_ship_lat.groupby(["common_name", "scientific_name", "species_abbreviation"]).agg({"count":"sum"})
.reset_index().sort_values(by="count", ascending=False).head(1))

,common_name,scientific_name,species_abbreviation,count
25,Wedge-tailed shearwater,Puffinus pacificus,PUFPAC,855.0


4) How many different types of birds were only ever seen in groups of 1?

In [134]:
(birds[birds["count"] == 1]
.groupby(["common_name", "scientific_name", "species_abbreviation"]).agg({"record_id":"count"}).shape[0])

287

5) How many penguins were seen? (Hint: there are many types of penguin)

In [137]:
penguins = birds[birds["common_name"].str.contains(pat="penguin", case=False)] # find the different types of penguin

In [147]:
penguins["count"].sum() #sum the column

158.0